## Running Materials Project Compatible workflows using VASP / DFT

As of early 2025, the majority of the calculations in the [Materials Project (MP) database](https://next-gen.materialsproject.org/materials) are performed with [PBE GGA](https://doi.org/10.1103/PhysRevLett.77.3865) and PBE+<i>U</i> density funcitonal theory (DFT) calculations, with about a quarter coming from [r<sup>2</sup>SCAN calculations](https://doi.org/10.1021/acs.jpclett.0c02405) instead.

In this example, you will use the Materials Project workflows and corrections/mixing schemes for [PBE and PBE+<i>U</i>](https://doi.org/10.1103/PhysRevB.84.045115), and for [r<sup>2</sup>SCAN](https://doi.org/10.1038/s41524-022-00881-w), to generate a partial compositional phase diagram for the V-O compositional space. The mixing and corrections schemes are used to better align PBE, PBE+<i>U</i>, and r<sup>2</sup>SCAN convex hulls, and to correct known errors in approximate DFT to predict more accurate enthalpies of formation.

<b>Note:</b> <i>The Materials Project is currently recomputing the entirety of its database with updated r<sup>2</sup>SCAN workflows. We will focus first on the current set of calculations, and mention these updated workflows at the end of the notebook.</i>

Let's start by investigating the input sets used by MP

In [1]:
from __future__ import annotations

from pymatgen.io.vasp.sets import MPRelaxSet, MPScanRelaxSet

print(  # noqa: T201
    "PBE GGA / +U INCAR:\n"
    + "\n".join(f"  {k}: {v}" for k, v in MPRelaxSet.CONFIG["INCAR"].items())
)
print(  # noqa: T201
    "r2SCAN meta-GGA INCAR:\n"
    + "\n".join(f"  {k}: {v}" for k, v in MPScanRelaxSet.CONFIG["INCAR"].items())
)

PBE GGA / +U INCAR:
  ALGO: FAST
  EDIFF_PER_ATOM: 5e-05
  ENCUT: 520
  IBRION: 2
  ISIF: 3
  ISMEAR: -5
  ISPIN: 2
  LASPH: True
  LDAU: True
  LDAUJ: {'F': {'Co': 0, 'Cr': 0, 'Fe': 0, 'Mn': 0, 'Mo': 0, 'Ni': 0, 'V': 0, 'W': 0}, 'O': {'Co': 0, 'Cr': 0, 'Fe': 0, 'Mn': 0, 'Mo': 0, 'Ni': 0, 'V': 0, 'W': 0}}
  LDAUL: {'F': {'Co': 2, 'Cr': 2, 'Fe': 2, 'Mn': 2, 'Mo': 2, 'Ni': 2, 'V': 2, 'W': 2}, 'O': {'Co': 2, 'Cr': 2, 'Fe': 2, 'Mn': 2, 'Mo': 2, 'Ni': 2, 'V': 2, 'W': 2}}
  LDAUTYPE: 2
  LDAUU: {'F': {'Co': 3.32, 'Cr': 3.7, 'Fe': 5.3, 'Mn': 3.9, 'Mo': 4.38, 'Ni': 6.2, 'V': 3.25, 'W': 6.2}, 'O': {'Co': 3.32, 'Cr': 3.7, 'Fe': 5.3, 'Mn': 3.9, 'Mo': 4.38, 'Ni': 6.2, 'V': 3.25, 'W': 6.2}}
  LDAUPRINT: 1
  LORBIT: 11
  LREAL: AUTO
  LWAVE: False
  NELM: 100
  NSW: 99
  PREC: Accurate
  SIGMA: 0.05
  MAGMOM: {'Ce': 5, 'Ce3+': 1, 'Co': 0.6, 'Co3+': 0.6, 'Co4+': 1, 'Cr': 5, 'Dy3+': 5, 'Er3+': 3, 'Eu': 10, 'Eu2+': 7, 'Eu3+': 6, 'Fe': 5, 'Gd3+': 7, 'Ho3+': 4, 'La3+': 0.6, 'Lu3+': 0.6, 'Mn': 5, 'Mn3+': 

While you can set up MP compatible VASP calculations by hand, it's usually best practice to use the pre-defined workflows in `atomate2`.
For PBE GGA/+<i>U</i> calculations, the workflow is `atomate2.vasp.flows.mp.MPGGADoubleRelaxStaticMaker`. For r<sup>2</sup>SCAN, the workflow is `atomate2.vasp.flows.mp.MPMetaGGADoubleRelaxStaticMaker`.

We start by pulling a few structures from the Materials Project API in the V-O system, importantly including the unaries.

In [5]:
from mp_api.client import MPRester

with MPRester() as mpr:
    docs = mpr.materials.summary.search(
        material_ids=["mp-146", "mp-12957", "mp-18937", "mp-541404"],
        fields=["material_id", "structure"],
    )
structures = {doc.material_id.string: doc.structure.to_primitive() for doc in docs}
for mp_id, structure in structures.items():
    print(structure.to(fmt="poscar", comment=f"{mp_id} {structure.formula}"))  # noqa: T201

Retrieving SummaryDoc documents:   0%|          | 0/4 [00:00<?, ?it/s]

mp-12957 O8
1.0
   3.1101070000000002    4.3774280000000001    0.0000000000000000
   3.1101070000000002   -4.3774280000000001    0.0000000000000000
   0.0000000000000000   -1.9083120000000000   -3.9533450000000001
O
8
direct
   0.2139109999999997    0.2734589999999988    0.8446089999999995 O
   0.2734589999999988    0.2139109999999999    0.1553910000000005 O
   0.7860889999999999    0.7265410000000010    0.1553910000000005 O
   0.7265410000000010    0.7860889999999999    0.8446089999999995 O
   0.7922520000000004    0.2077479999999996    0.8216799999999995 O
   0.2077479999999996    0.7922520000000004    0.1783200000000006 O
   0.8517779999999995    0.1482220000000004    0.1326409999999996 O
   0.1482220000000005    0.8517779999999995    0.8673590000000004 O

mp-18937 V4 O6
1.0
   2.4648877418460842    4.3700725910251847    0.0000000000000000
   2.4648877418460842   -4.3700725910251847    0.0000000000000000
   0.0000000000000000   -2.9541189453930183   -4.6676819387169539
V O
4 6
direc

### If you don't have access to VASP, or just want to run the tutorial example:

All test data is provided with this example (including raw VASP input and output), just run the following code block:

In [2]:
from mock_vasp import TEST_DIR, mock_vasp
from monty.serialization import loadfn

tasks = loadfn(TEST_DIR / "vasp" / "mp_phase_diagram_tutorial" / "MP-example-V-O-phase-diagram.json.gz")

structures = {mp_id: task_block["PBEsol Structure Optimization"].input.structure for mp_id, task_block in tasks.items()}

Now let's actually set up the workflows! For these calculations, we will adjust "parallelization settings" to ensure that our calculations run quickly.
If you are not familiar with these settings in VASP, it's best to [read the manual section in VASP](https://www.vasp.at/wiki/index.php/Category:Parallelization).

If you run VASP on CPU nodes on a cluster/HPC, you want to modify:
- [NCORE](https://www.vasp.at/wiki/index.php/NCORE): how many individual cores work on a single orbital
- [KPAR](https://www.vasp.at/wiki/index.php/KPAR): how many <i>k</i> points are treated in parallel

If you run VASP on GPU nodes on a cluster/HPC, you will modify:
- [NSIM](https://www.vasp.at/wiki/index.php/NSIM): parallelization over bands for certain optimizers
- KPAR
- NCORE = 1

To do the latter part, we will use the powerups in `atomate2`. We will also save our results to JSON-format output using `monty`.

For parallelization settings, I'm running these on GPU nodes, so I set `NCORE = 1`, `KPAR = 8` (using 8 individual GPUs), and setting `NSIM=8` as a reasonable value that does not increase the memory demands too much.'

Note that we also update the names of the jobs and their metadata to make it easier to query output later when we build the phase diagram.

In [3]:
from jobflow import Flow, run_locally, JobStore
from maggma.stores import MemoryStore
from monty.serialization import dumpfn

from atomate2.vasp.flows.mp import (
    MPGGADoubleRelaxStaticMaker,
    MPMetaGGADoubleRelaxStaticMaker,
)
from atomate2.vasp.powerups import update_user_incar_settings
from mock_vasp import TEST_DIR, mock_vasp

jobs = []
for mp_id, structure in structures.items():
    pbe_job = MPGGADoubleRelaxStaticMaker().make(structure)
    pbe_job.append_name(f" {mp_id}")
    pbe_job.update_metadata({"mp_id": mp_id})

    r2scan_job = MPMetaGGADoubleRelaxStaticMaker().make(structure)
    r2scan_job.append_name(f" {mp_id}")
    pbe_job.update_metadata({"mp_id": mp_id})

    jobs.extend([pbe_job, r2scan_job])

all_jobs = Flow(jobs)
all_jobs = update_user_incar_settings(all_jobs, {"NCORE": 1, "KPAR": 8, "NSIM": 8})

In [4]:
test_data_dir = TEST_DIR / "vasp" / "mp_phase_diagram_tutorial"

ref_paths = {
    k : test_data_dir / v for k, v in {
        "MP GGA static mp-12957": "MP_GGA_static_mp-12957.json.lzma",
        "MP GGA relax 2 mp-18937": "MP_GGA_relax_2_mp-18937.json.lzma",
        "MP GGA relax 1 mp-18937": "MP_GGA_relax_1_mp-18937.json.lzma",
        "MP pre-relax 1 mp-146": "MP_pre-relax_1_mp-146.json.lzma",
        "MP meta-GGA relax 2 mp-146": "MP_meta-GGA_relax_2_mp-146.json.lzma",
        "MP pre-relax 1 mp-541404": "MP_pre-relax_1_mp-541404.json.lzma",
        "MP meta-GGA static mp-146": "MP_meta-GGA_static_mp-146.json.lzma",
        "MP GGA static mp-146": "MP_GGA_static_mp-146.json.lzma",
        "MP meta-GGA static mp-12957": "MP_meta-GGA_static_mp-12957.json.lzma",
        "MP meta-GGA static mp-541404": "MP_meta-GGA_static_mp-541404.json.lzma",
        "MP meta-GGA relax 2 mp-541404": "MP_meta-GGA_relax_2_mp-541404.json.lzma",
        "MP GGA relax 2 mp-146": "MP_GGA_relax_2_mp-146.json.lzma",
        "MP meta-GGA relax 2 mp-12957": "MP_meta-GGA_relax_2_mp-12957.json.lzma",
        "MP pre-relax 1 mp-12957": "MP_pre-relax_1_mp-12957.json.lzma",
        "MP GGA relax 1 mp-12957": "MP_GGA_relax_1_mp-12957.json.lzma",
        "MP GGA static mp-541404": "MP_GGA_static_mp-541404.json.lzma",
        "MP GGA relax 1 mp-146": "MP_GGA_relax_1_mp-146.json.lzma",
        "MP GGA static mp-18937": "MP_GGA_static_mp-18937.json.lzma",
        "MP GGA relax 2 mp-12957": "MP_GGA_relax_2_mp-12957.json.lzma",
        "MP pre-relax 1 mp-18937": "MP_pre-relax_1_mp-18937.json.lzma",
        "MP meta-GGA relax 2 mp-18937": "MP_meta-GGA_relax_2_mp-18937.json.lzma",
        "MP GGA relax 1 mp-541404": "MP_GGA_relax_1_mp-541404.json.lzma",
        "MP meta-GGA static mp-18937": "MP_meta-GGA_static_mp-18937.json.lzma",
        "MP GGA relax 2 mp-541404": "MP_GGA_relax_2_mp-541404.json.lzma"
    }.items()
}

job_store = JobStore(MemoryStore(),additional_stores = {"data": MemoryStore()})

with mock_vasp(ref_paths) as vasp:
    responses = run_locally(
        all_jobs,
        store=job_store,
        create_folders=True,
        ensure_success = True,
    )

2025-04-04 11:22:41,744 INFO Started executing jobs locally
2025-04-04 11:22:41,754 INFO Starting job - MP GGA relax 1 mp-146 (270a409c-9c04-42cf-8740-ee21a5f133b7)
2025-04-04 11:22:41,875 INFO Finished job - MP GGA relax 1 mp-146 (270a409c-9c04-42cf-8740-ee21a5f133b7)
2025-04-04 11:22:41,875 WARNING Response.stored_data is not supported with local manager.
2025-04-04 11:22:41,876 INFO Starting job - MP GGA relax 2 mp-146 (f25b3296-43ee-4388-8f99-bd5aa7716772)


/Users/aaronkaplan/miniconda3/envs/matpes_mlff/lib/python3.11/site-packages/jobflow/core/flow.py:423: UserWarning: Some jobs are not connected, their ordering may be random
  for node in itergraph(graph):
/Users/aaronkaplan/miniconda3/envs/matpes_mlff/lib/python3.11/site-packages/pymatgen/io/vasp/sets.py:483: BadInputSetWarning: Relaxation of likely metal with ISMEAR < 0 (-5). See VASP recommendations on ISMEAR for metals (https://www.vasp.at/wiki/index.php/ISMEAR).
  incar=self.incar,
/Users/aaronkaplan/miniconda3/envs/matpes_mlff/lib/python3.11/site-packages/pymatgen/io/vasp/outputs.py:1280: UserWarning: No POTCAR file with matching TITEL fields was found in

  if potcar := self.get_potcars(path):
/Users/aaronkaplan/miniconda3/envs/matpes_mlff/lib/python3.11/site-packages/pymatgen/io/vasp/outputs.py:1291: UserWarning: No POTCAR file with matching TITEL fields was found in

  potcar = self.get_potcars(path)
/Users/aaronkaplan/Library/CloudStorage/Dropbox/mp_soft_engr/software/atomate2

2025-04-04 11:22:42,024 INFO Finished job - MP GGA relax 2 mp-146 (f25b3296-43ee-4388-8f99-bd5aa7716772)
2025-04-04 11:22:42,024 WARNING Response.stored_data is not supported with local manager.
2025-04-04 11:22:42,025 INFO Starting job - MP GGA static mp-146 (1d6f9ee8-8681-4612-8c6f-ffc3faf87f74)
2025-04-04 11:22:42,161 INFO Finished job - MP GGA static mp-146 (1d6f9ee8-8681-4612-8c6f-ffc3faf87f74)
2025-04-04 11:22:42,162 WARNING Response.stored_data is not supported with local manager.
2025-04-04 11:22:42,162 INFO Starting job - MP pre-relax 1 mp-146 (1d51936f-6595-4fdd-a29b-b66381650dd8)


/Users/aaronkaplan/Library/CloudStorage/Dropbox/mp_soft_engr/software/atomate2/src/atomate2/common/files.py:268: UserWarning: /private/var/folders/mv/ltq1lcxn15d0wngg9vlgd1jw0000gn/T/tmpzod5hxrv/job_2025-04-04-18-22-42-025044-71396/POTCAR.spec is not gzipped, skipping...
  file_client.gunzip(directory / file, host=host, force=force)


2025-04-04 11:22:42,319 INFO Finished job - MP pre-relax 1 mp-146 (1d51936f-6595-4fdd-a29b-b66381650dd8)
2025-04-04 11:22:42,319 WARNING Response.stored_data is not supported with local manager.
2025-04-04 11:22:42,319 INFO Starting job - MP meta-GGA relax 2 mp-146 (278defc1-6cd6-4fe9-b965-85fc86361414)


...

2025-04-04 11:22:58,206 INFO Finished job - MP GGA static mp-541404 (0b1dca20-3ca8-4a9e-92c6-c20f0dfc6489)
2025-04-04 11:22:58,207 WARNING Response.stored_data is not supported with local manager.
2025-04-04 11:22:58,207 INFO Starting job - MP pre-relax 1 mp-541404 (00abc3cf-76aa-47cb-bf50-64fe4562fb3f)
2025-04-04 11:23:00,400 INFO Finished job - MP pre-relax 1 mp-541404 (00abc3cf-76aa-47cb-bf50-64fe4562fb3f)
2025-04-04 11:23:00,401 WARNING Response.stored_data is not supported with local manager.
2025-04-04 11:23:00,402 INFO Starting job - MP meta-GGA relax 2 mp-541404 (936d79f8-e6f6-4011-a498-c4ca305c2024)


/Users/aaronkaplan/Library/CloudStorage/Dropbox/mp_soft_engr/software/atomate2/src/atomate2/common/files.py:268: UserWarning: /private/var/folders/mv/ltq1lcxn15d0wngg9vlgd1jw0000gn/T/tmpzod5hxrv/job_2025-04-04-18-23-00-401775-60635/POTCAR.spec is not gzipped, skipping...
  file_client.gunzip(directory / file, host=host, force=force)


2025-04-04 11:23:04,524 INFO Finished job - MP meta-GGA relax 2 mp-541404 (936d79f8-e6f6-4011-a498-c4ca305c2024)
2025-04-04 11:23:04,525 WARNING Response.stored_data is not supported with local manager.
2025-04-04 11:23:04,525 INFO Starting job - MP meta-GGA static mp-541404 (08b37541-9119-4b2a-9bd5-ce385bad2ce2)


/Users/aaronkaplan/Library/CloudStorage/Dropbox/mp_soft_engr/software/atomate2/src/atomate2/common/files.py:268: UserWarning: /private/var/folders/mv/ltq1lcxn15d0wngg9vlgd1jw0000gn/T/tmpzod5hxrv/job_2025-04-04-18-23-04-525432-40746/POTCAR.spec is not gzipped, skipping...
  file_client.gunzip(directory / file, host=host, force=force)


2025-04-04 11:23:07,627 INFO Finished job - MP meta-GGA static mp-541404 (08b37541-9119-4b2a-9bd5-ce385bad2ce2)
2025-04-04 11:23:07,627 WARNING Response.stored_data is not supported with local manager.
2025-04-04 11:23:07,627 INFO Finished executing jobs locally


If you ran this locally, you can collect the output of the jobs from the local directory structure. We use the formula of the original structures and `calc_type` (name of the functional used plus the type of job performed, like a static or a relaxation) to distinguish jobs. We will use `tasks` to plot phase diagrams.

In [ ]:
from pathlib import Path

from emmet.core.tasks import TaskDoc

formula_to_mpid = {s.formula: mpid for mpid, s in structures.items()}
task_list = [TaskDoc.from_directory(p) for p in Path(".").glob("job_*") if p.is_dir()]
tasks = {mpid: {} for mpid in structures}
for task in task_list:
    mpid = formula_to_mpid.get(task.structure.formula)
    ct = task.calc_type.value
    if ct in tasks[mpid]:
        nmatch = len([k for k in tasks[mpid] if ct in k])
        ct += f" {nmatch + 1}"
    tasks[mpid][ct] = task

dumpfn(tasks, "V-O_MP_task_docs.json.gz")

Alternatively, you could use the following code to obtain the output using the job names directly. It's a good exercise to see how you can use `all_output` in the following phase diagram plots.

In [ ]:
from atomate2.utils.testing import get_job_uuid_name_map

job_uuid_to_name = get_job_uuid_name_map(all_jobs)

all_output = {name: responses[uuid][1].output for uuid, name in job_uuid_to_name.items()}

Last, and this is the way you should go in high-throughput, we can query the job store for output. The contents of the job store are "deserialized," so we reconstruct them with their document model, `emmet.core.tasks.TaskDoc`.

Note that we only need the static calculations (with `task_type` of `GGA Static`, `GGA+U Static`, and `r2SCAN Static`) to build the phase diagram, as these most accurately capture the total energy. That's because of various electronic structure calculation reasons (e.g., as the lattice changes during a geomtry optimization, the <i>k</i> point grid may no longer have high-enough density, or it may represent the wrong lattice symmetry).

In [5]:
from emmet.core.tasks import TaskDoc

all_output = {}
for mp_id in structures:
    all_output[mp_id] = {}
    for entry in job_store.query({"metadata.mp_id": mp_id, "name": {"$regex" : "static"}}):
        all_output[mp_id][entry["output"]["calc_type"]] = TaskDoc(**entry["output"])

In the dictionary of `tasks` that we just built, the static calculations (`GGA Static`, `GGA+U Static`, and `r2SCAN Static`) should most accurately capture the total energy. That's because of various electronic structure calculation reasons (e.g., as the lattice changes during a geomtry optimization, the <i>k</i> point grid may no longer have high-enough density, or it may represent the wrong lattice symmetry).

We will use these static calculations to build a compositional phase diagram. First, we have to apply mixing and correction scheme corrections to these tasks. These are applied by `pymatgen.entries.mixing_scheme.MaterialsProjectDFTMixingScheme` - we only have to pass a small representation of each calculation called a `ComputedStructureEntry`, to the mixer, and it applies all empirical corrections / functional mixing adjustments.

<b>Note:</b> `MaterialsProjectDFTMixingScheme` will give sensible output only if you have complete phase space information (elemental unary compounds and relevant intermediates). We will cover the case of applying corrections without adjusting the PBE and r<sup>2</sup>SCAN hulls in a moment.

To best demonstrate how the mixing corrections work, we will purposefully omit one of the r<sup>2</sup>SCAN calculations.

In [6]:
from pymatgen.entries.mixing_scheme import MaterialsProjectDFTMixingScheme

static_entries = []
for mpid, calcs in all_output.items():
    for calc_type, task_doc in calcs.items():
        if mpid == "mp-18937" and calc_type.lower() == "r2scan static":
            continue

        if "static" in calc_type.lower():
            cse = task_doc.structure_entry
            cse.entry_id = f"{cse.parameters['run_type']} {mpid}"
            cse.data["material_id"] = mpid
            static_entries.append(cse)

mixer = MaterialsProjectDFTMixingScheme()
corrected_entries = mixer.process_entries(static_entries)

for ce in corrected_entries:
    print(ce.entry_id, ce.energy_adjustments)  # noqa: T201

GGA mp-12957 []
GGA+U mp-541404 [CompositionEnergyAdjustment(name='MP2020 anion correction (oxide)', value=-5.5, uncertainty=0.016, description='Composition-based energy adjustment', generated_by='MaterialsProject2020Compatibility'), CompositionEnergyAdjustment(name='MP2020 GGA/GGA+U mixing correction (V)', value=-6.8, uncertainty=0.0256, description='Composition-based energy adjustment', generated_by='MaterialsProject2020Compatibility')]
GGA mp-146 []
GGA+U mp-18937 [CompositionEnergyAdjustment(name='MP2020 anion correction (oxide)', value=-4.12, uncertainty=0.012, description='Composition-based energy adjustment', generated_by='MaterialsProject2020Compatibility'), CompositionEnergyAdjustment(name='MP2020 GGA/GGA+U mixing correction (V)', value=-6.8, uncertainty=0.0256, description='Composition-based energy adjustment', generated_by='MaterialsProject2020Compatibility')]


/Users/aaronkaplan/miniconda3/envs/matpes_mlff/lib/python3.11/site-packages/pymatgen/entries/mixing_scheme.py:183: UserWarning: r2SCAN entries do not form a complete PhaseDiagram.
  mixing_state_data = self.get_mixing_state_data(entries_type_1 + entries_type_2)


Now let's actually plot the phase diagram! Note that the one calculation where we purposefully skipped the r<sup>2</sup>SCAN calculation, `mp-18937`, is represented by a PBE+<i>U</i> calculation. You can see this by hovering your cursor over that particular data point.

Try to compare your phase diagram to the one that [MP calculates](https://next-gen.materialsproject.org/phasediagram).

What happens if you do not skip this calculation? How does the convex hull change?

In [7]:
from pymatgen.analysis.phase_diagram import PhaseDiagram

pd = PhaseDiagram(corrected_entries)
fig = pd.get_plot()
fig.update_layout(width=1000, height=700, plot_bgcolor="white", paper_bgcolor="white")

To apply just corrections/preliminary mixing to the PBE and PBE+<i>U</i> entries, you would use `pymatgen.entries.compatibility.MaterialsProject2020Compatibility`. This is useful if you want to compare to older MP data in the [legacy site](https://legacy.materialsproject.org/). The correction scheme will raise a `CompatibilityError` if you attempt to apply it to non-PBE/+<i>U</i> entries.

In [ ]:
from pymatgen.entries.compatibility import MaterialsProject2020Compatibility
from pymatgen.entries.computed_entries import ComputedStructureEntry

compat_scheme = MaterialsProject2020Compatibility()
gga_gga_u_corrected_entries = []
for mpid, calcs in tasks.items():
    for calc_type, task_doc in calcs.items():
        if all(kwd in calc_type.lower() for kwd in ("gga", "static")):
            cse_dict = task_doc.structure_entry.as_dict()
            cse_dict["energy_adjustments"] = compat_scheme.get_adjustments(
                task_doc.structure_entry
            )
            corrected_cse = ComputedStructureEntry.from_dict(cse_dict)
            print(  # noqa: T201
                mpid, corrected_cse.formula, calc_type, corrected_cse.energy_adjustments
            )

### Note on revised MP input sets / workflows

The Materials Project recently undertook a significant [benchmarking effort](https://github.com/materialsproject/foundation/pull/26) to verify the accuracy of its DFT calculations. That ultimately led to the construction of new r<sup>2</sup>SCAN input sets, which are currently being used to recompute all entries in MP.

While these data aren't currently publicly available, if you want to use these newer input sets / workflows, we list them below:

```python
from pymatgen.io.vasp.sets import MP24RelaxSet
from atomate2.vasp.flows.mp import MP24DoubleRelaxStaticMaker
```

Then you would use `MP24DoubleRelaxStaticMaker` in place of `MPMetaGGARelaxMaker`.